# Julia for maths

- Goal: Think of Julia as a **tool** for doing mathematics

- Julia was designed with the explicit goal of being productive for "technical computing"

- Tension between mathematics and computer science


- Problem: Mathematics is (are) not precise enough

- Computing is *more precise* than math!

- Cf. [this great talk about the Lean theorem prover](https://www.youtube.com/watch?v=Dp-mQ3HxgDE)



1. Julia as math

    - Syntax: Short function definitions, unicode

    - Functions returning functions

    - Generic functions
    

2. Generic programming

    - Types

    - When to use BigFloats

    - Passing vectors and functions as arguments
    
    - Unpacking vectors 

    - Positional vs. keyword arguments
    
    - Wrap state into an object
    
    - Named tuples


3. Vectors

    - Mathematical "vectors" vs computational "arrays"

    - True mathematical vectors: Define a type

    - StaticArrays: `SVector` and `MVector`
    
    - Array comprehensions 
    
    - Broadcasting: `.`
    
4. Iterations

    - `x[n]` versus `x_new`

Theorem prover: https://leanprover.github.io/



## Syntax

In [5]:
f(x) = 2x

f (generic function with 1 method)

In [6]:
f(α, x) = α*x

f (generic function with 2 methods)

In [7]:
r = 3
r² = r * r

9

## Functions are first class objects

In [8]:
lorenz(a, b, x, y) = a + b + x + y

lorenz (generic function with 1 method)

In [11]:
function lorenz(a, b, x, y)
    return 2a + b + x + y
end

lorenz (generic function with 1 method)

In [12]:
lorenz(a, b) = a + b

lorenz (generic function with 2 methods)

In [13]:
lorenz(1, 2)

3

In [14]:
lorenz(a::Integer, b::Float64) = a + 2b

lorenz (generic function with 3 methods)

In [15]:
lorenz(1, 2)

3

In [16]:
lorenz(1.0, 2.0)

3.0

In [17]:
lorenz(1, 2.0)

5.0

In [18]:
methods(lorenz)

# 3 methods for generic function "lorenz":
[1] lorenz(a::Integer, b::Float64) in Main at In[14]:1
[2] lorenz(a, b) in Main at In[12]:1
[3] lorenz(a, b, x, y) in Main at In[11]:2

Logistic map:

In [20]:
logistic(r, x) = r*x * (1 - x)

logistic (generic function with 1 method)

In [21]:
logistic(r) = x -> logistic(r, x)  # "the logistic map with parameter r

logistic (generic function with 2 methods)

In [22]:
function fixed_point(f, x0)
    x = x0
    
    for i in 1:10
        x = f(x)
    end
    
    return x
end

fixed_point (generic function with 1 method)

In [24]:
g(x) = 2.1*x*(1-x)

g (generic function with 1 method)

In [25]:
fixed_point(g, 0.6)

0.5238095238213468

In [27]:
fixed_point(logistic(2.1), 0.6)

0.5238095238213468

In [29]:
fixed_point(x -> 1 + 0.5*x, 0.6)

1.9986328125

In [30]:
logistic(0.5)

#5 (generic function with 1 method)

In [ ]:
function lorenz(a, xx)
#     x = xx[1]
#     y = xx[2]
#     z = xx[3]
    
    x, y, z = xx   # "unpacking"
    
    return x*y - z
end
    
    

In [31]:
lorenz(3.1, [1.0, 2.0, 3.0])

MethodError: MethodError: no method matching +(::Float64, ::Array{Float64,1})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(::Float64, !Matched::Float64) at float.jl:401
  +(::AbstractFloat, !Matched::Bool) at bool.jl:106
  ...

In [32]:
Meta.@lower x, y, z = xx

:($(Expr(:thunk, CodeInfo(
    @ none within `top-level scope'
1 ─ %1  = Base.indexed_iterate(xx, 1)
│   %2  = Core.getfield(%1, 1)
│         x = %2
│         #s1 = Core.getfield(%1, 2)
│   %5  = Base.indexed_iterate(xx, 2, #s1)
│   %6  = Core.getfield(%5, 1)
│         y = %6
│         #s1 = Core.getfield(%5, 2)
│   %9  = Base.indexed_iterate(xx, 3, #s1)
│   %10 = Core.getfield(%9, 1)
│         z = %10
└──       return xx
))))

## Generic programming

In [34]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1260


In [35]:
f(x) = sin(3x + exp(2x))

f (generic function with 2 methods)

In [36]:
@btime f(3.1)

  7.242 ns (0 allocations: 0 bytes)


-0.5694063164846805

In [37]:
@btime f(big(3.1))

  5.817 μs (27 allocations: 984 bytes)


-0.5694063164846744544130095747756931941742993496004926505391221726206440072041555

In [38]:
setprecision(BigFloat, 106)

106

In [39]:
@btime f(big(3.1))

  3.836 μs (27 allocations: 760 bytes)


-0.569406316484674454413009574773421

In [44]:
function compare(exact, numerical)
    return abs(exact - numerical)
end

compare (generic function with 2 methods)

In [45]:
compare(sqrt(2), sqrt(big(2)))

9.66729331345291345105469972976695e-17

In [46]:
x = sqrt(big(2))

1.41421356237309504880168872420969

In [47]:
compare(sqrt(2), x)

9.66729331345291345105469972976695e-17

In [48]:
sqrt(2)

1.4142135623730951

In [49]:
typeof(ans)

Float64

## Generic programming

Write functions that work for any type 

In [51]:
function h(f, x)  # avoid restricting to certain type
    f(x) + f(f(x))
end

h (generic function with 1 method)

In [52]:
h(3, 4)

MethodError: MethodError: objects of type Int64 are not callable

In [53]:
h(sin, 10)

-1.0616018783541432

In [54]:
h(sin, [3, 4])

MethodError: MethodError: no method matching sin(::Array{Int64,1})
Closest candidates are:
  sin(!Matched::BigFloat) at mpfr.jl:744
  sin(!Matched::Missing) at math.jl:1157
  sin(!Matched::Complex{Float16}) at math.jl:1105
  ...

In [55]:
function h2(f::Function, x)  # avoid restricting to certain type
    f(x) + f(f(x))
end

h2 (generic function with 1 method)

In [56]:
f(x, y) = x^2 + y

f (generic function with 2 methods)

In [57]:
x = [3, 4, 5]

3-element Array{Int64,1}:
 3
 4
 5

In [58]:
sin(x)

MethodError: MethodError: no method matching sin(::Array{Int64,1})
Closest candidates are:
  sin(!Matched::BigFloat) at mpfr.jl:744
  sin(!Matched::Missing) at math.jl:1157
  sin(!Matched::Complex{Float16}) at math.jl:1105
  ...

In [59]:
[3, "hello", -3.5]

3-element Array{Any,1}:
  3
   "hello"
 -3.5

In [60]:
v = [3, 4, 5]

3-element Array{Int64,1}:
 3
 4
 5

In [61]:
v + v

3-element Array{Int64,1}:
  6
  8
 10

In [62]:
3*v

3-element Array{Int64,1}:
  9
 12
 15

In [63]:
push!(v, 10)

4-element Array{Int64,1}:
  3
  4
  5
 10

In [64]:
v

4-element Array{Int64,1}:
  3
  4
  5
 10

In [67]:
sin.(v)  # broadcasting

4-element Array{Float64,1}:
  0.1411200080598672
 -0.7568024953079282
 -0.9589242746631385
 -0.5440211108893698

In [68]:
map(sin, v)

4-element Array{Float64,1}:
  0.1411200080598672
 -0.7568024953079282
 -0.9589242746631385
 -0.5440211108893698

In [69]:
sin.(cos.(v))

4-element Array{Float64,1}:
 -0.8360218615377305
 -0.6080830096407656
  0.2798733507685274
 -0.7440230792707043

In [70]:
@. sin(cos(v))

4-element Array{Float64,1}:
 -0.8360218615377305
 -0.6080830096407656
  0.2798733507685274
 -0.7440230792707043

In [71]:
[sin(cos(x)) for x in v]

4-element Array{Float64,1}:
 -0.8360218615377305
 -0.6080830096407656
  0.2798733507685274
 -0.7440230792707043

In [76]:
[sin(cos(x)) for x in v if x > 5]

1-element Array{Float64,1}:
 -0.7440230792707043

In [78]:
sum( sin(cos(x)) for x in v if x > 4 )

-0.46414972850217695

In [79]:
using LinearAlgebra

In [80]:
[1, 2] × [3, 4]

DimensionMismatch: DimensionMismatch("cross product is only defined for vectors of length 3")

In [81]:
[1, 2, 4] × [3, 4, 6]

3-element Array{Int64,1}:
 -4
  6
 -2

Tension between mathematical and container vectors: issue #4774 in JuliaLang GitHub repo -- talk by Jiahao Chen at JuliaCon 2017

In [82]:
v = [3, 4, 5]

3-element Array{Int64,1}:
 3
 4
 5

In [83]:
transpose(v)

1×3 Transpose{Int64,Array{Int64,1}}:
 3  4  5

In [84]:
v'

1×3 Adjoint{Int64,Array{Int64,1}}:
 3  4  5

In [85]:
v' * v

50

In [ ]:
using Pkg  
Pkg.add("StaticArrays")  ## only once!

In [87]:
using StaticArrays  ## every time you use -- in every session

In [88]:
struct FixedSizeVector
    data::NTuple{2,Float64}
end
    

In [90]:
v = FixedSizeVector( (3, 4) )

FixedSizeVector((3.0, 4.0))

In [91]:
v = FixedSizeVector( (3, 4, 5) )

MethodError: MethodError: Cannot `convert` an object of type 
  Tuple{Int64} to an object of type 
  Tuple{}
Closest candidates are:
  convert(::Type{Tuple{}}, ::Tuple{Any,Vararg{Any,N} where N}) at essentials.jl:306
  convert(::Type{Tuple{}}, !Matched::Tuple{}) at essentials.jl:305
  convert(::Type{Tuple{}}, !Matched::CartesianIndices{0,R} where R<:Tuple{}) at multidimensional.jl:290
  ...

In [94]:
(3, 4) + (5, 6)

MethodError: MethodError: no method matching +(::Tuple{Int64,Int64}, ::Tuple{Int64,Int64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529

In [95]:
(3, 4) .+ (5, 6)

(8, 10)

In [ ]:
struct ParamFixedSizeVector{N}
    data::NTuple{N,Float64}
end

In [96]:
using StaticArrays

In [97]:
v = SVector(3, 4)

2-element SArray{Tuple{2},Int64,1,2} with indices SOneTo(2):
 3
 4

In [98]:
v + v

2-element SArray{Tuple{2},Int64,1,2} with indices SOneTo(2):
 6
 8

In [99]:
w = SVector(3, 4, 5)

3-element SArray{Tuple{3},Int64,1,3} with indices SOneTo(3):
 3
 4
 5

In [100]:
v + w

DimensionMismatch: DimensionMismatch("Sizes (Size(2,), Size(3,)) of input arrays do not match")

In [101]:
v

2-element SArray{Tuple{2},Int64,1,2} with indices SOneTo(2):
 3
 4

In [102]:
v[1]

3

In [103]:
v[1] = 10

ErrorException: setindex!(::SArray{Tuple{2},Int64,1,2}, value, ::Int) is not defined.

In [ ]:
function iterate(f, x)
    xs = zeros(10)
    for i in 1:10
        xs[n+1] = f(xs[n])
    end
    return xs
end

In [ ]:
function iterate(f, x)
    xs = [x]
    for i in 1:10
        x_new = f(x_old)
        
        push!(xs, x_new)
        x_old = x_new
    end
    return xs
end